<table>
    <tr>
        <td><img src="./imagenes/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp>
                <h1 style="color:blue;text-align:center">Inteligencia Artificial</h1
            </tp>
            <tp>
                <p style="font-size:150%;text-align:center">Métodos de Diferencia Temporal</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivo <a class="anchor" id="inicio"></a>

En este notebook veremos una manera de implementar los métodos tabulares de Diferencia Temporal, los cuales usaremos para resolver algunos ambientes de tarea. Realizaremos múltiples pruebas para evaluar las bondades de cada algoritmo sobre distintos entornos.

Este notebook está basado en las presentación de Sanghi (2021), capítulo 5 y sus [notebooks](https://github.com/Apress/deep-reinforcement-learning-python); y de Sutton R., & Barto, A., (2018), capítulo 6. 

[Ir al ejercicio 1](#ej1)

# Dependencias

Al iniciar el notebook o reiniciar el kerner se pueden cargar todas las dependencias de este notebook corriendo las siguientes celdas. Este también es el lugar para instalar las dependencias que podrían hacer falta.

**De Python:**

In [ ]:
# En linux o mac
#!pip3 install -r requirements.txt
#!pip3 install gymnasium[toy-text]

# En windows
#!python -m pip install -r requirements.txt
#!python -m pip installgymnasium[toy-text]

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored, cprint

**Del notebook:**

In [ ]:
from ambientes import GridworldEnv, CliffworldEnv
from agents import Agent
from algoritmos import *
from utils import Episode, Experiment
from plot_utils import PlotGridValues, Plot
from tests import *

# Secciones

Desarrollaremos la explicación de la siguiente manera:

* [Evaluación de una política](#eval)
* [Métodos de control](#control)

# Evaluación de una política <a class="anchor" id="eval"></a>
    
([Volver al inicio](#inicio))

Recordemos que lo primero que hacemos para resolver un ambiente de tarea es considerar el problema de la evaluación. Aquí lo que queremos es determinar el valor de un estado dada una política. En otras palabras, queremos estimar el valor esperado de la recompensa total descontada si seguimos una política a partir de un estado $s$. Esto para cada uno de los estados $s$ del ambiente de tarea. Examinaremos el método de diferencia temporal para atacar el problema y usaremos como ejemplo el entorno del GridWorld.

Comenzamos definiendo una política sobre un mundo de rejilla de tamaño $4\times 4$ (observe que esta es la política que trabajamos en el notebook anterior):

In [ ]:
shape = (4,4)
env = GridworldEnv(shape=shape)
policy = ([env.NORTH] + [env.EAST]*(shape[0] - 2) + [env.SOUTH]) * shape[1]
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
pp.plot_policy(policy)

Los valores de estado ya los conocemos. Usaremos el método de programación dinámica para obtenerlos:

In [ ]:
V = policy_eval(env, policy)
pp.plot_policy_and_values(policy, V)

En el método de evaluación de diferencia temporal no requerimos esperar a que el episodio entero termine antes de actualizar nuestros estimados de valor para los estados visitados. En lugar de ello, en cada ronda estamos actualizando el estimado del valor del estado que acabamos de visitar, usando un bootstrap con los valores en memoria. El pesudocódigo del algoritmo es el siguiente:

<img src="./imagenes/td_evaluationb.png" width="500"/>

<a class="anchor" id="ej1"></a>**Ejercicio 1:** 

([Próximo ejercicio](#ej2))

Implemente las líneas 5 a 8 del pseudocódigo anterior en el codigo siguiente:

In [ ]:
def td_0_evaluation(env, policy:np.array, alfa:float=0.1, gama:float=1, max_iter:int=500, max_steps=1000, V:np.array=None) -> np.array:
    '''
    Método de diferencia temporal para estimar el valor de los 
    estados de un MDP generando una muestra de episodios con base en una política dada. 
    Input:
        - env, un ambiente con atributos nA, nS, shape 
               y métodos reset(), step()
        - policy, una política determinista, policy[state] = action
        - alfa, real con el parámetro de step-size
        - gama, con el parámetro de factor de descuento
        - max_iter, entero con la cantidad máxima de episodios
        - max_steps, entero con la cantidad máxima de pasos
        - Opcional: V, un np.array que por cada s devuelve su valor estimado
    Output:
        - V, un np.array que por cada s devuelve su valor estimado
    '''
    if V is None:
        V = np.zeros(env.nS)
    for _ in range(max_iter):
        state = np.random.randint(env.nS)
        env.state = state
        done = False
        counter = 0
        while not done:
            pass
            # AQUÍ SU CÓDIGO
            
            # HASTA AQUÍ SU CÓDIGO
            counter += 1
            if counter > max_steps:
                break
    return V 


Compruebe su respuesta corriendo la siguiente celda:

In [ ]:
np.random.seed(3)
V = td_0_evaluation(env, policy)
VV = np.flipud(np.reshape(V, shape))
test = np.array([[ 0.,         -4.36081499, -3.97308306, -2.99943748],
                 [-0.99993144, -3.77424704, -2.99432731, -1.99999999],
                 [-1.99314462, -2.7058518,  -1.99561459, -1.   ],
                 [-2.85262296, -1.7706891, -0.99695675,  0.   ]])
assert(np.all(np.isclose(VV, test)))
print('¡Correcto!')

---

## Métodos de control <a class="anchor" id="control"></a>
    
([Volver al inicio](#inicio))

Resolver un entorno consiste en encontrar la política óptima que debe seguir el agente para maximizar su utilidad. En el mundo de la rejilla, la política óptima es la siguiente:

In [ ]:
shape = (4,4)
env = GridworldEnv(shape=shape)
policy = value_iteration(env, discount_factor=1, theta=0.01, verbose=0)
print('Política óptima:')
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
pp.plot_policy(policy)

Para poder encontrar la política óptima, el método de Diferencia Temporal guarda en memoria un estimado del valor estado-acción para la política, pero cada vez que cambian su estimado, también mejora la política mediante una $\epsilon$ mejora. 

$$
\pi_{k+1}(a|s) = \begin{cases}
1-\epsilon &\text{si }a=\text{arg}\!\max_{a'} Q_{\pi_k}(s,a') \cr
\frac{\epsilon}{\#\text{acciones}-1} &\text{en otro caso}
\end{cases}
$$

### La clase Episode

Para correr las simulaciones de manera ordenada y sencilla, hemos implementado la clase `Episode`, que se encuentran en el módulo `utils.py`. Veamos un algoritmo aleatorio actuando en el entorno del mundo de la rejilla:

In [ ]:
# Create environment
shape = (3,3)
env = GridworldEnv(shape=shape)
# Create agent
parameters = {\
    'nS': np.prod(shape),\
    'nA': 4,\
    'gamma': 1,\
    'epsilon': 0.1,\
}
agent = Agent(parameters=parameters)
# Create episode
episode = Episode(environment=env, \
                  env_name='GW', \
                  agent=agent, \
                  model_name='Random', \
                  num_rounds=3, \
                )
# Run and show information
df = episode.run(verbose=4)

Observe que hemos corrido el método `run()`, el cual corre solo un episodio, con la opción verbose = 4. Esta opción hace que el método imprima la información de cada ronda del episodio. Cuando no deseemos dicha información, podemos poner verbose = 0 (u omitir este argumento del todo, pues es opcional con valor por defecto 0) para evitar demoras innecesarias en la obtención de los datos.

Veamos ahora una presentación gráfica de la evolución de la recompensa total por varios episodios:

In [ ]:
# Create environment
shape = (4,4)
env = GridworldEnv(shape=shape)
# Create agent
parameters = {\
    'nS': np.prod(shape),\
    'nA': 4,\
    'gamma': 1,\
    'epsilon': 0.1,\
}
agent = Agent(parameters=parameters)
# Create episode
episode = Episode(environment=env, \
                  env_name='GW', \
                  agent=agent, \
                  model_name='Random', \
                  num_rounds=100
                )
# Train agent
df = episode.simulate(num_episodes=50, verbose=0)
# Plot rewards
Plot(df).plot_rewards()

Aquí vemos la evolución de la recompensa por 50 episodios, cada uno de máximo 100 rondas. Para ello hemos usado el método `simulate()`.

Podemos inspeccionar la política del agente al visualizar su atributo `policy`:

In [ ]:
p = episode.agent.policy
policy = [np.argmax(p[s,]) for s in range(env.nS)]
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
print('Política del agente:')
pp.plot_policy(policy)

**La clase Experiment**

Una manera de usar la clase `Experiment` es para testear el agente ya entrenado y sin que este tenga que hacer exploración. Podemos usar el método `run_experiment()` sobre el agente y dibujar un histograma de la recompensa total por episodio:

In [ ]:
# Create experiment
experiment = Experiment(environment=env,\
                        env_name='GW', \
                        num_rounds=10, \
                        num_episodes=10, \
                        num_simulations=10)
# Test agent
agents = experiment.run_experiment(agents=[agent],\
                                  names=['Random'], \
                                  measures=['hist_reward'], \
                                  learn=False)

La clase `Experiment` tiene varias funcionalidades adicionales que exploraremos más adelante.

### Temporal Difference control <a class="anchor" id="TDcontrol"></a>
    
([Volver a Control](#control))

Vamos a implementar dos agentes. Uno con la regla de aprendizaje SARSA y otro con la regla Q-learning.

<a class="anchor" id="ej2"></a>**Ejercicio 2:** 

([Anterior ejercicio](#ej1)) ([Próximo ejercicio](#ej3))

Implemente el agente SARSA de acuerdo al pseudo código del agente SARSA:

<img src="./imagenes/sarsa_agent.png" width="500"/>

Use la siguiente celda para implementar el agente. El énfasis es en implementar la linea 5 encontrando el update mediante bootstrapping (estimate), el error de diferencia temporal (delta) y la actualización del valor previo al moverlo en dirección de delta una fracción alfa.

In [ ]:
class SARSA(Agent) :
    '''
    Implements a SARSA learning rule.
    '''

    def __init__(self, parameters:dict):
        super().__init__(parameters)
        self.alpha = self.parameters['alpha']
        self.debug = False
   
    def update(self, next_state, reward, done):
        '''
        Agent updates its model.
        '''
        # obtain previous state
        state = self.states[-1]
        # obtain previous action
        action = self.actions[-1]
        # Get next_action
        next_action = self.make_decision()
        # Find bootstrap
        estimate = ... # recompensa más descuento por valor del siguiente estado
        # Obtain delta
        delta = ... # Diferencia temporal: estimado menos valor del estado actual
        # Update Q value
        prev_Q = self.Q[state, action]
        self.Q[state, action] = ... # Actualizar en la dirección de delta por una fracción alfa
        # Update policy
        self.update_policy(state)
        if self.debug:
            print('')
            print(dash_line)
            print(f'Learning log:')
            print(f'state:{state}')
            print(f'action:{action}')
            print(f'reward:{reward}')
            print(f'estimate:{estimate}')
            print(f'Previous Q:{prev_Q}')
            print(f'delta:{delta}')
            print(f'New Q:{self.Q[state, action]}')


Corra la siguiente celda para verificar su implementación.

In [ ]:
shape = (3,4)
# Create agent
parameters = {\
    'nS': np.prod(shape),\
    'nA': 4,\
    'gamma': 1,\
    'epsilon': 0.1,\
    'alpha': 0.1, \
}
agent_SARSA = SARSA(parameters=parameters)
test = test_sarsa(agent_SARSA)
if test:
    cprint(colored('¡Test superado!', 'green'))
else:
    cprint(colored('¡Implementación incorrecta!', 'red'))

---

<a class="anchor" id="ej3"></a>**Ejercicio 3:** 

([Anterior ejercicio](#ej2)) ([Próximo ejercicio](#ej4))

Implemente el agente con la regla de aprendizaje Q-learning, de acuerdo al siguiente pseudocódigo:

<img src="./imagenes/q_learning_agent.png" width="450"/>


In [ ]:
class Q_learning(Agent) :
    '''
    Implements a Q-learning rule.
    '''

    def __init__(self, parameters:dict):
        super().__init__(parameters)
        self.alpha = self.parameters['alpha']
        self.debug = False
   
    def update(self, next_state, reward, done):
        '''
        Agent updates its model.
        '''
        # obtain previous state
        state = ... # Aquí estado previo
        # obtain previous action
        action = self.actions[-1]
        # Find bootstrap
        maxQ = self.max_Q(next_state) 
        estimate = ... # Calcula el estimado
        # Obtain delta
        delta = ... # Calcula el delta
        # Update Q value
        prev_Q = self.Q[state, action]
        self.Q[state, action] = ... # Actualiza el valor
        # Update policy
        self.update_policy(...) # Actualizar la política en el estado        
        if self.debug:
            print('')
            print(dash_line)
            print(f'Learning log:')
            print(f'state:{state}')
            print(f'action:{action}')
            print(f'reward:{reward}')
            print(f'estimate:{estimate}')
            print(f'Previous Q:{prev_Q}')
            print(f'delta:{delta}')
            print(f'New Q:{self.Q[state, action]}') 

Corra la siguiente celda para verificar su implementación.

In [ ]:
shape = (3,4)
# Create agent
parameters = {\
    'nS': np.prod(shape),\
    'nA': 4,\
    'gamma': 1,\
    'epsilon': 0.1,\
    'alpha': 0.1, \
}
agent_Q = Q_learning(parameters=parameters)
test = test_q(agent_Q)
if test:
    cprint(colored('¡Test superado!', 'green'))
else:
    cprint(colored('¡Implementación incorrecta!', 'red'))

---

<a class="anchor" id="ej4"></a>**Ejercicio 4:** 

([Anterior ejercicio](#ej3)) ([Próximo ejercicio](#ej5))

Compare el desempeño del agente SARSA con el del agente Q-learning en el entorno de la caminata por el acantilado (implementado en la clase `CliffworldEnv` del módulo `ambientes`). Observe que este ejemplo fue discutido en clase, en el cual se mencionó que el Q-learning no toma en cuenta los deslices ocacionales de la política $\epsilon$-avara, mientras que SARSA sí.

Use las siguientes especificaciones:

* número máximo de rondas: 200
* número de episodios: 500
* número de simulaciones: 10

---

Podemos ver las políticas resultantes para los agentes al correr las siguientes celdas:

**Nota:** Las siguientes celdas solo funcionan después de haber realizado el ejercicio 4.

In [ ]:
# Create environment
shape = (3,4)
env = CliffworldEnv(shape=shape)
shape = (3,4)
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
sarsa = agents[0]
p = sarsa.policy
policy = [np.argmax(p[s,]) for s in range(env.nS)]
policy = np.flipud(np.reshape(policy, shape))
pp.plot_policy(policy)

In [ ]:
# Create environment
shape = (3,4)
env = CliffworldEnv(shape=shape)
shape = (3,4)
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
q_agent = agents[1]
p = q_agent.policy
policy = [np.argmax(p[s,]) for s in range(env.nS)]
policy = np.flipud(np.reshape(policy, shape))
pp.plot_policy(policy)

Durante el proceso de aprendizaje, queremos que el agente pruebe distintos cursos de acción, de manera tal que tenga una mayor confianza en que está llegando a una política óptima. No obstante, a la hora de poner a marchar al agente en producción, queremos que el agente tenga su mejor desempeño. Para ello, necesitamos poner su parámetro $\epsilon$ en 0. 

Vamos a comparar el desempeño de los agentes SARSA y Q-learning en su desempeño óptimo, sin exploración. 

In [ ]:
# Shut down exploration
agents[0].epsilon = 0
agents[1].epsilon = 0
for s in range(env.nS):
    agents[0].update_policy(s)
    agents[1].update_policy(s)

In [ ]:
# Create experiment
experiment = Experiment(environment=env,\
                 env_name='Cliff', \
                 num_rounds=100, \
                 num_episodes=100, \
                 num_simulations=1)
# Use stored agents to run test
experiment.run_experiment(
                agents=agents,\
                names=['SARSA', 'Q_learning'], \
                measures=['hist_reward'],\
                learn=False)
print('¡Listo!')

---

# En este notebook usted aprendió

* Cómo implementar la regla de evaluación de una política usando diferencia temporal.
* Cómo implementar los métodos de mejoramiento de política SARSA y Q-learning.

# Bibliografía

([Volver al inicio](#inicio))

Shanghi, N. (2021) Deep Reinforcement Learning with Python: With PyTorch, TensorFlow and OpenAI Gym. Apress. 

Sutton R., & Barto, A., (2015) Reinforcement Learning: An Introduction, 2nd Edition. A Bradford Book. Series: Adaptive Computation and Machine Learning series. 

Winder, P., (2021) Reinforcement Learning: Industrial Applications of Intelligent Agents. O’Relly.